In [1]:
import random
from animalai.environment import AnimalAIEnvironment
from wrappers import CustomUnityToGymWrapper

In [ ]:
aai_seed = 0
port = 5005 + random.randint(
    0, 1000
)  # uses a random port to avoid problems if a previous version exits slowly
env_path = "/Users/Ile-Maurice/Desktop/MacOS/MacOS"
configuration_file = "./animal_ai_environments/yroom.yaml"
watch = True

aai_env = AnimalAIEnvironment(
    file_name=env_path,  # Path to the environment
    seed=aai_seed,  # seed for the pseudo random generators
    arenas_configurations=configuration_file,
    play=False,  # note that this is set to False for training
    base_port=port,  # the port to use for communication between python and the Unity environment
    inference=watch,  # set to True if you want to watch the agent play
    useCamera=True,  # set to False if you don't want to use the camera (no visual observations)
    resolution=84,
    useRayCasts=False,  # set to True if you want to use raycasts
    no_graphics=False,  # set to True if you don't want to use the graphics ('headless' mode)
    timescale=1,
)

env = CustomUnityToGymWrapper(
    aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True
)  # the wrapper for the environment

# # fix for error generated by the gym wrapper on line 241, python3.10/site-packages/mlagents_envs/rpc_utils.py in _observation_to_np_array
# # use img = img.reshape(obs.shape)

In [ ]:
print(env.action_space)
# 0 - nothing
# 1 - rotate right by 6 degrees
# 2 - rotate left by 6 degrees
# 3 - accelerate forward
# 4 - accelerate forward and rotate CW by 6 degrees
# 5 - accelerate forward and rotate CCW by 6 degrees
# 6 - accelerate backward
# 7 - accelerate backward and rotate CW by 6 degrees
# 8 - accelerate backward and rotate CCW by 6 degrees


print(env.observation_space) # see python3.10/site-packages/animalai/environment.py, line 202
# Box(84, 84, 3) - rgb image
# Box(-inf, inf, (7,)) - (health; v1, v2, v3; p1, p2, p3)

In [ ]:
env.reset()

In [ ]:
# for i in range(60):
#     obs, reward, done, info = env.step(1)
#     print("health:", obs[1][1])
#     print("vel:", obs[1][1:4]) # (v?, v?, v forward/backard relative to starting orientation)
#     print("pos:", obs[1][4:7]) # (p?, p?, p forward/backard relative to starting orientation)

# for i in range(30):
#     obs, reward, done, info = env.step(4)
#     obs, reward, done, info = env.step(7)
#     print("health:", obs[1][1])
#     print("vel:", obs[1][1:4])
#     print("pos:", obs[1][4:7])

obs, reward, done, info = env.step(3)
print("obs[0] shape", obs[0].shape)
print("obs[0][0] ", obs[0][0])
print("health:", obs[1][1])
print("vel:", obs[1][1:4])
print("pos:", obs[1][4:7])

In [ ]:
env.close()

In [24]:
import random
from animalai.environment import AnimalAIEnvironment
from wrappers import CustomUnityToGymWrapper
from animalai_agent import AnimalAIVectorhashAgent
from vectorhash import build_vectorhash_architecture
import random

### vhash
shapes = [(3,3,4), (4,4,5)]
model = build_vectorhash_architecture(shapes, N_h=1200, input_size=84*84, initalization_method="by_sparsity", shift="conv")




### animalai
aai_seed = 0
port = 5005 + random.randint(
    0, 1000
)  # uses a random port to avoid problems if a previous version exits slowly
env_path = "/Users/Ile-Maurice/Desktop/MacOS/MacOS"
configuration_file = "./animal_ai_environments/yroom.yaml"
watch = True

aai_env = AnimalAIEnvironment(
    file_name=env_path,  # Path to the environment
    seed=aai_seed,  # seed for the pseudo random generators
    arenas_configurations=configuration_file,
    play=False,  # note that this is set to False for training
    base_port=port,  # the port to use for communication between python and the Unity environment
    inference=False,  # set to True if you want to watch the agent play
    useCamera=True,  # set to False if you don't want to use the camera (no visual observations)
    resolution=84,
    useRayCasts=False,  # set to True if you want to use raycasts
    no_graphics=False,  # set to True if you don't want to use the graphics ('headless' mode)
    timescale=0.1,
)

env = CustomUnityToGymWrapper(
    aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True
)  # the wrapper for the environment





#### agent
agent = AnimalAIVectorhashAgent(model, env)
agent.vectorhash.certainty = 2

by_sparsity
module shapes:  [(3, 3, 4), (4, 4, 5)]
N_g     :  116
N_patts :  2880
N_h     :  1200


I0000 00:00:1743977024.262150 7214941 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


[UnityMemory] Configuration Parameters - Can be set up in boot.config
    "memorysetup-bucket-allocator-granularity=16"
    "memorysetup-bucket-allocator-bucket-count=8"
    "memorysetup-bucket-allocator-block-size=4194304"
    "memorysetup-bucket-allocator-block-count=1"
    "memorysetup-main-allocator-block-size=16777216"
    "memorysetup-thread-allocator-block-size=16777216"
    "memorysetup-gfx-main-allocator-block-size=16777216"
    "memorysetup-gfx-thread-allocator-block-size=16777216"
    "memorysetup-cache-allocator-block-size=4194304"
    "memorysetup-typetree-allocator-block-size=2097152"
    "memorysetup-profiler-bucket-allocator-granularity=16"
    "memorysetup-profiler-bucket-allocator-bucket-count=8"
    "memorysetup-profiler-bucket-allocator-block-size=4194304"
    "memorysetup-profiler-bucket-allocator-block-count=1"
    "memorysetup-profiler-allocator-block-size=16777216"
    "memorysetup-profiler-editor-allocator-block-size=1048576"
    "memorysetup-temp-allocator-siz

/Users/Ile-Maurice/Desktop/rl-research-main/rl-research/vectorhash/vectorhash.py:74: UserWarning: Using a target size (torch.Size([1, 1200])) that is different to the input size (torch.Size([1200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  torch.nn.functional.mse_loss(h, h_from_s),
/Users/Ile-Maurice/Desktop/rl-research-main/rl-research/vectorhash/vectorhash.py:81: UserWarning: Using a target size (torch.Size([1, 1200])) that is different to the input size (torch.Size([1200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  torch.nn.functional.mse_loss(h, h_from_s_denoised),
/Users/Ile-Maurice/Desktop/rl-research-main/rl-research/vectorhash/vectorhash.py:97: UserWarning: Using a target size (torch.Size([1, 7056])) that is different to the input size (torch.Size([7056])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the sam

In [20]:
agent.step(2)

[tensor([9.9999e-01, 2.6822e-06, 2.6822e-06]), tensor([9.9999e-01, 2.6822e-06, 6.3949e-12, 2.6822e-06])]
[tensor([9.9999e-01, 2.6822e-06, 2.6822e-06]), tensor([9.9999e-01, 2.6822e-06, 6.3949e-12, 2.6822e-06])]
[tensor([3.1974e-12, 2.6822e-06, 1.0000e+00, 2.2235e-18]), tensor([9.9396e-25, 2.2235e-18, 3.1974e-12, 2.6822e-06, 1.0000e+00])]
tensor(5.9195e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
tensor(5.9195e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
tensor(3.0356e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
tensor(5.9195e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
tensor(5.9195e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
tensor(3.0356e-06)
tensor([-4, -3, -2, -1,  0,  1,  2,  3,  4,  5], dtype=torch.int32)
info for each h directly after learning it
h max, min, mean tensor(2.3305) tensor(1.2079) tensor(1.7225)
h_from_s max, min, mean tensor(2.9

In [21]:
agent.calculate_position_err()

(tensor(1.1921e-05), tensor(1.1921e-05), tensor(5.9605e-06))

In [25]:
def test_path(path):
    # takes in as input list of actions
    errs = []
    for i in path:
        agent.step(i)
        errs.append(agent.calculate_position_err())
    return errs

In [22]:
env.close()